<a href="https://colab.research.google.com/github/yfpang7/dataScience_projects/blob/main/requests_xpaths_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from lxml import html
from bs4 import BeautifulSoup
import polars as pl

In [2]:
# test url
autotrade_url = 'https://www.autotrader.co.uk/car-search?advertising-location=at_cars&channel=cars&distance=&homeDeliveryAdverts=include&make=MINI&model=Hatch&postcode=CB1%203FL&price-to=7000&sort=relevance&year-from=2016'

url = 'https://www.dtc-lease.nl/'

right_move = 'https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index=0&sortType=2&channel=BUY&transactionType=BUY&displayLocationIdentifier=CB1.html&maxPrice=700000'



In [3]:
# get the rightmove url

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/127.0.0.0 Safari/537.36"
}

response = requests.get(right_move, headers=headers)

response.status_code

200

In [4]:
# parse the html
tree = html.fromstring(response.text)

# get the href of each property on the link
links = tree.xpath('//a[contains(@data-testid , "property-details-lozenge")]/@href')

links[:5]

['/properties/163672550#/?channel=RES_BUY',
 '/properties/166327667#/?channel=RES_BUY',
 '/properties/166046009#/?channel=RES_BUY',
 '/properties/163461650#/?channel=COM_BUY',
 '/properties/166587485#/?channel=RES_BUY']

In [5]:
# get all the urls for all the available pages on rightmove
urls =  [
    f'https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index={i}&sortType=2&channel=BUY&transactionType=BUY&displayLocationIdentifier=CB1.html&maxPrice=700000'
    for i in range(0, 1500, 24)
]

In [6]:
relative_urls = []
# get the href in each of the links
for url in urls:
  r = requests.get(url, headers=headers)
  tree = html.fromstring(r.text)
  print(f'Successfully requests: {url}')

  # get the the href from each property on each page
  links = tree.xpath('//a[contains(@data-testid , "property-details-lozenge")]/@href')
  # break if there is no more links
  if not links:
    break
  # append the href or the relative urls into an array
  relative_urls.extend(links)

Successfully requests: https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index=0&sortType=2&channel=BUY&transactionType=BUY&displayLocationIdentifier=CB1.html&maxPrice=700000
Successfully requests: https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index=24&sortType=2&channel=BUY&transactionType=BUY&displayLocationIdentifier=CB1.html&maxPrice=700000
Successfully requests: https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index=48&sortType=2&channel=BUY&transactionType=BUY&displayLocationIdentifier=CB1.html&maxPrice=700000
Successfully requests: https://www.rightmove.co.uk/property-for-sale/find.html?useLocationIdentifier=true&locationIdentifier=OUTCODE%5E409&radius=10.0&_includeSSTC=on&index=72&sortType=2&chan

In [7]:
# check the first 5 relative urls
relative_urls[:5]

['/properties/166046180#/?channel=RES_BUY',
 '/properties/166327667#/?channel=RES_BUY',
 '/properties/166046009#/?channel=RES_BUY',
 '/properties/163461650#/?channel=COM_BUY',
 '/properties/166587485#/?channel=RES_BUY']

In [8]:
# combine the base url with the relative urls
# create the full list
full_urls = [f"{'https://www.rightmove.co.uk'}{relative}" for relative in relative_urls]

In [9]:
# check the full links if they work
full_urls[:5]

['https://www.rightmove.co.uk/properties/166046180#/?channel=RES_BUY',
 'https://www.rightmove.co.uk/properties/166327667#/?channel=RES_BUY',
 'https://www.rightmove.co.uk/properties/166046009#/?channel=RES_BUY',
 'https://www.rightmove.co.uk/properties/163461650#/?channel=COM_BUY',
 'https://www.rightmove.co.uk/properties/166587485#/?channel=RES_BUY']

In [10]:
# defining the xpaths to extract each data
xpaths = {
    'address' : '//div[contains(@class, "h3U6cGyEUf76tvCpYisik")]/h1/text()',
    'price' : '//div[contains(@class, "_1gfnqJ3Vtd1z40MlC0MzXu")]/span/text()',
    'property_type' : '(//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[1]/p/text()',
    'bedrooms' : '(//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[2]/p/text()',
    'bathrooms' : '(//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[3]/p/text()',
    'size' : '(//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[4]/p/text()',
    'tenure' : '(//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[5]/p/text()',
    'council_tax' : '(//dd[contains(@class, "_2zXKe70Gdypr_v9MUDoVCm")])[1]/text()',
    'parking' : '(//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[2]/dd/span/text()',
    'garden' : '(//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[3]/dd/span/text()',
    'accessibility' : '(//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[4]/dd/span/text()',

}

In [11]:
[f"{key} : {value}" for key, value in xpaths.items()]

['address : //div[contains(@class, "h3U6cGyEUf76tvCpYisik")]/h1/text()',
 'price : //div[contains(@class, "_1gfnqJ3Vtd1z40MlC0MzXu")]/span/text()',
 'property_type : (//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[1]/p/text()',
 'bedrooms : (//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[2]/p/text()',
 'bathrooms : (//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[3]/p/text()',
 'size : (//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[4]/p/text()',
 'tenure : (//div[contains(@class, "_3gIoc-NFXILAOZEaEjJi1n")]/dd/span)[5]/p/text()',
 'council_tax : (//dd[contains(@class, "_2zXKe70Gdypr_v9MUDoVCm")])[1]/text()',
 'parking : (//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[2]/dd/span/text()',
 'garden : (//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[3]/dd/span/text()',
 'accessibility : (//div[contains(@class, "_9u6R9n55iQlZi-JF6H59W")])[4]/dd/span/text()']

In [12]:
# create a list to store all scraped data
all_data = []

for url in full_urls:
  detail_response = requests.get(url, headers=headers)
  detail_tree = html.fromstring(detail_response.text)
  # print(f'Successfully requested: {url}')

  # create a dictionary to store the scrapped data
  # for one property
  row= {'url' : url}

  # go through each of the field and its corresponding XPath
  for field in xpaths:
    # print(field)
    # field = the key ('address', 'etc)
    # xpath = is the value from the key
    xpath = xpaths[field]
    # print(xpath)
    result = detail_tree.xpath(xpath)

    # if we get any results, use the first one
    if result:
      row[field] = result[0].strip()
    else:
      row[field] = None

  # add this row to our data list
  all_data.append(row)

data = pl.DataFrame(all_data)

# show the data
data.head()


url,address,price,property_type,bedrooms,bathrooms,size,tenure,council_tax,parking,garden,accessibility
str,str,str,str,str,str,str,str,str,str,str,str
"""https://www.rightmove.co.uk/pr…","""Stirling Road, Northstowe, C…","""£469,995""","""Semi-Detached""","""4""","""Ask developer""","""Ask developer""",null,"""Ask developer""","""Ask developer""","""Yes""","""Ask developer"""
"""https://www.rightmove.co.uk/pr…","""Highfield Road, Impington""","""£700,000""","""Detached""","""3""","""1""","""Ask agent""","""Freehold""","""Band: E""","""Yes""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""Auckland Road, Cambridge""","""£700,000""","""End of Terrace""","""3""","""1""","""915 sq ft""","""Freehold""","""Band: D""","""Permit""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""London Road, Little Chesterfor…","""£700,000""","""Land""","""4""","""3""","""30,492 sq ft""",null,null,null,null,null
"""https://www.rightmove.co.uk/pr…","""High Street, Teversham""","""£700,000""","""Detached""","""3""","""2""","""1,600 sq ft""","""Freehold""","""Band: F""","""Yes""","""Yes""","""Ask agent"""


In [13]:
# write the data
# !pip install xlsxwriter
# data.write_excel('rightmove_CB1_within10miles.xlsx')

In [14]:
# read from the last scrape
# !pip install fastexcel
# data = pl.read_excel('/content/rightmove_CB1_within10miles.xlsx')

In [15]:
data.head()

url,address,price,property_type,bedrooms,bathrooms,size,tenure,council_tax,parking,garden,accessibility
str,str,str,str,str,str,str,str,str,str,str,str
"""https://www.rightmove.co.uk/pr…","""Stirling Road, Northstowe, C…","""£469,995""","""Semi-Detached""","""4""","""Ask developer""","""Ask developer""",null,"""Ask developer""","""Ask developer""","""Yes""","""Ask developer"""
"""https://www.rightmove.co.uk/pr…","""Highfield Road, Impington""","""£700,000""","""Detached""","""3""","""1""","""Ask agent""","""Freehold""","""Band: E""","""Yes""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""Auckland Road, Cambridge""","""£700,000""","""End of Terrace""","""3""","""1""","""915 sq ft""","""Freehold""","""Band: D""","""Permit""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""London Road, Little Chesterfor…","""£700,000""","""Land""","""4""","""3""","""30,492 sq ft""",null,null,null,null,null
"""https://www.rightmove.co.uk/pr…","""High Street, Teversham""","""£700,000""","""Detached""","""3""","""2""","""1,600 sq ft""","""Freehold""","""Band: F""","""Yes""","""Yes""","""Ask agent"""


In [16]:
# start cleaning the data
clean_data = (
    data
    .with_columns(
        pl.col('price').str.replace_all(r'£|,', '').cast(pl.Float64),
        pl.col('bedrooms').cast(pl.Categorical,),
        pl.col('bathrooms').cast(pl.Categorical),
        # \d* : one or more digits
        # \.?  = optinal decimals
        # \d* = zero or more digits after decimals
        # () = capture the group for extract the match
        pl.col('size').str.to_lowercase().str.replace_all(r'sq ft|,', '').str.replace_all(' ', '').str.extract(r'(\d+\.?\d*)').cast(pl.Float64),
        pl.col('council_tax').str.replace_all('Band: ', '').str.replace_all(r'Ask developer|Ask agent', '').replace('', None)
    )
)

In [17]:
clean_data.head()

url,address,price,property_type,bedrooms,bathrooms,size,tenure,council_tax,parking,garden,accessibility
str,str,f64,str,cat,cat,f64,str,str,str,str,str
"""https://www.rightmove.co.uk/pr…","""Stirling Road, Northstowe, C…",469995.0,"""Semi-Detached""","""4""","""Ask developer""",null,null,null,"""Ask developer""","""Yes""","""Ask developer"""
"""https://www.rightmove.co.uk/pr…","""Highfield Road, Impington""",700000.0,"""Detached""","""3""","""1""",null,"""Freehold""","""E""","""Yes""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""Auckland Road, Cambridge""",700000.0,"""End of Terrace""","""3""","""1""",915.0,"""Freehold""","""D""","""Permit""","""Yes""","""Ask agent"""
"""https://www.rightmove.co.uk/pr…","""London Road, Little Chesterfor…",700000.0,"""Land""","""4""","""3""",30492.0,null,null,null,null,null
"""https://www.rightmove.co.uk/pr…","""High Street, Teversham""",700000.0,"""Detached""","""3""","""2""",1600.0,"""Freehold""","""F""","""Yes""","""Yes""","""Ask agent"""
